In [1]:
# jupyter测试
print('hello Jupyter~~~!')

hello Jupyter~~~!


In [2]:
# 获取mnist数据集，并加载到内存中
import mxnet as mx
mnist = mx.test_utils.get_mnist()

In [3]:
# 初始化MNIST数据集的数据迭代器，一个用于训练一个用于测试
batch_size = 100
train_iter = mx.io.NDArrayIter(mnist['train_data'],mnist['train_label'],batch_size,shuffle=True)
val_iter = mx.io.NDArrayIter(mnist['test_data'],mnist['test_label'],batch_size)


In [4]:
data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="tanh")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="tanh")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="tanh")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=10)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')

In [5]:
#使用SGD梯度下降方法开始训练
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())
# train with the same
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 100),
                num_epoch=10)

INFO:root:Epoch[0] Batch [100]	Speed: 2171.86 samples/sec	accuracy=0.108515


INFO:root:Epoch[0] Batch [200]	Speed: 2229.95 samples/sec	accuracy=0.114900


INFO:root:Epoch[0] Batch [300]	Speed: 2206.86 samples/sec	accuracy=0.110800


INFO:root:Epoch[0] Batch [400]	Speed: 2234.56 samples/sec	accuracy=0.111800


INFO:root:Epoch[0] Batch [500]	Speed: 2216.49 samples/sec	accuracy=0.115800


...

INFO:root:Epoch[9] Batch [400]	Speed: 2201.55 samples/sec	accuracy=0.990200


INFO:root:Epoch[9] Batch [500]	Speed: 2282.53 samples/sec	accuracy=0.990600


INFO:root:Epoch[9] Train-accuracy=0.992424


INFO:root:Epoch[9] Time cost=26.984


INFO:root:Epoch[9] Validation-accuracy=0.988300


In [9]:
#使用模型进行预测
test_iter = mx.io.NDArrayIter(mnist['test_data'], None, batch_size)
prob = lenet_model.predict(test_iter)
test_iter = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
# predict accuracy for lenet
acc = mx.metric.Accuracy()
lenet_model.score(test_iter, acc)
print("Accuracy score is %f" % (acc.get()[1]))
assert acc.get()[1] > 0.98

Accuracy score is 0.988300
